In [2]:
import pandas as pd
import numpy as np
import re

em_data_filepath='emdat_public_2022_08_11_query_uid-IKGfZc.xlsx'

dfb = pd.read_excel(em_data_filepath, engine='openpyxl',skiprows=6,sheet_name = 'emdat data')



flood_grp=['Drought']

#dfb1_ea=dfb1[dfb1['Country'].str.contains(ea_cntr)]
dfb_dr=dfb[dfb['Disaster Type'].isin(flood_grp)]

ea_cntr=['Djibouti', 'Eritrea','Ethiopia','Kenya','Rwanda', 'Somalia','South Sudan','Sudan (the)','Tanzania, United Republic of']

#dfb1_ea=dfb1[dfb1['Country'].str.contains(ea_cntr)]
dfb_dr_ea=dfb_dr[dfb_dr['Country'].isin(ea_cntr)]

dfb_dr_ea.info()

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 77 to 4601
Data columns (total 50 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Dis No                                     62 non-null     object 
 1   Year                                       62 non-null     int64  
 2   Seq                                        62 non-null     int64  
 3   Glide                                      23 non-null     object 
 4   Disaster Group                             62 non-null     object 
 5   Disaster Subgroup                          62 non-null     object 
 6   Disaster Type                              62 non-null     object 
 7   Disaster Subtype                           62 non-null     object 
 8   Disaster Subsubtype                        0 non-null      object 
 9   Event Name                                 0 non-null      object 
 10  Country                  

In [3]:
import pandas as pd

dfb_dr_path='ea_drought_location_v1.xlsx'

dfb_dr=pd.read_excel(dfb_dr_path, engine='openpyxl')
dfb_dr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   62 non-null     int64 
 1   Dis No       62 non-null     object
 2   Location     60 non-null     object
 3   Adm Level    48 non-null     object
 4   Admin1 Code  31 non-null     object
 5   Admin2 Code  24 non-null     object
 6   geob         14 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.5+ KB


# replacing GAUL Admin1 Code into GEOB admin2 codes

In [4]:
dfb_dr['gaul_adm1_list']=dfb_dr['Admin1 Code'].str.split(';')


em_dat_path='./'


gaul_adm1_dict_db=pd.read_csv(f'{em_dat_path}em_dat_ea_gaul_geob_adm1_codeshare.csv')

gaul_adm1_dict = dict(zip(gaul_adm1_dict_db.gaul_adm1_code, gaul_adm1_dict_db.geob_adm1_shapeID))


dfb_dr_adm1=dfb_dr[dfb_dr['gaul_adm1_list'].notna()]

dfb_dr_adm1['gaul_adm1_list_int']=dfb_dr_adm1['gaul_adm1_list'].apply(lambda x: [int(i) for i in x])


dfb_dr_adm1['gaul_adm1_geob'] = dfb_dr_adm1['gaul_adm1_list_int'].apply(lambda x: [gaul_adm1_dict[i] for i in x])

#dfb_dr_adm1

dfb_dr_adm1_a=dfb_dr_adm1[['Dis No','gaul_adm1_geob']]

dfb_dr_adm1_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 13 to 59
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Dis No          31 non-null     object
 1   gaul_adm1_geob  31 non-null     object
dtypes: object(2)
memory usage: 744.0+ bytes


/tmp/ipykernel_45/2101602999.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm1['gaul_adm1_list_int']=dfb_dr_adm1['gaul_adm1_list'].apply(lambda x: [int(i) for i in x])
/tmp/ipykernel_45/2101602999.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm1['gaul_adm1_geob'] = dfb_dr_adm1['gaul_adm1_list_int'].apply(lambda x: [gaul_adm1_dict[i] for i in x])


## replace adm1 code with adm2 sub codes

In [6]:
import geopandas as gp
import numpy as np

geob_adm1_adm2_filepath='/home/ea_shapefiles/'


geob_db=gp.read_file(f'{geob_adm1_adm2_filepath}ea_adm2_geoboundaries_adm1_code.shp')


#https://stackoverflow.com/a/54137935/2501953


geob_db1=geob_db[['adm1_shape','shapeID']]
#geob_db2=geob_db1.set_index('adm1_shape')


#geob_db_dict=geob_db1['adm1_shape'].groupby(level=0).apply(lambda x: x.to_dict('shapeID')).to_dict()

geob_db2=geob_db1.groupby('adm1_shape').agg({'shapeID':lambda x: list(x)})
geob_db3=geob_db2.reset_index()
geob_db3.columns=['gaul_adm1_geob','shapeID']

geob_db3_dict = dict(zip(geob_db3.gaul_adm1_geob, geob_db3.shapeID))


dfb_dr_adm1_a['geob_adm2'] = dfb_dr_adm1_a['gaul_adm1_geob'].apply(lambda x: [geob_db3_dict[i] for i in x])

def flatten(l):
    return [item for sublist in l for item in sublist]

aa_list=[]
for idx,row in dfb_dr_adm1_a.iterrows():
    aa=flatten(row['geob_adm2'])
    #print(aa)
    #dfb_dr_adm1_a.at[idx,'geob_adm2_list'] = list(flatten(row['geob_adm2']))
    aa_list.append(aa)
    aa=[]
    
dfb_dr_adm1_a['geob_adm2_list']=aa_list
    
dfb_dr_part1=dfb_dr_adm1_a[['Dis No','geob_adm2_list']]
dfb_dr_part1

/tmp/ipykernel_45/2330873680.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm1_a['geob_adm2'] = dfb_dr_adm1_a['gaul_adm1_geob'].apply(lambda x: [geob_db3_dict[i] for i in x])
/tmp/ipykernel_45/2330873680.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm1_a['geob_adm2_list']=aa_list


,Dis No,geob_adm2_list
13,1999-9388-ERI,"[ERI-ADM2-3_0_0-B20, ERI-ADM2-3_0_0-B21, ERI-A..."
14,1999-9388-ETH,"[ETH-ADM2-3_0_0-B5, ETH-ADM2-3_0_0-B33, ETH-AD..."
16,1999-9388-DJI,"[DJI-ADM2-3_0_0-B10, DJI-ADM2-3_0_0-B7, DJI-AD..."
17,1999-9388-SDN,"[SDN-ADM2-3_0_0-B78, SDN-ADM2-3_0_0-B79, SDN-A..."
18,1999-9388-RWA,"[RWA-ADM2-3_0_0-B1, RWA-ADM2-3_0_0-B2, RWA-ADM..."
19,1999-9388-SOM,"[SOM-ADM2-3_0_0-B5, SOM-ADM2-3_0_0-B14, SOM-AD..."
20,2003-9772-ETH,"[ETH-ADM2-3_0_0-B70, ETH-ADM2-3_0_0-B71, ETH-A..."
21,2004-9288-KEN,"[KEN-ADM2-3_0_0-B52, KEN-ADM2-3_0_0-B12, KEN-A..."
23,2003-9651-TZA,"[TZA-ADM2-3_0_0-B12, TZA-ADM2-3_0_0-B35, TZA-A..."
24,2004-9085-SOM,"[SOM-ADM2-3_0_0-B31, SOM-ADM2-3_0_0-B34, SOM-A..."


# replacing GAUL Admin2 Code into GEOB admin2 codes

In [7]:
dfb_dr['gaul_adm2_list']=dfb_dr['Admin2 Code'].str.split(';')


em_dat_path='./'


gaul_adm2_dict_db=pd.read_excel(f'{em_dat_path}em_dat_ea_gaul_geob_adm2_codeshare_v1.xlsx', engine='openpyxl')

gaul_adm2_dict = dict(zip(gaul_adm2_dict_db.gaul_adm2_code, gaul_adm2_dict_db.geob_adm2_shapeID))


dfb_dr_adm2=dfb_dr[dfb_dr['gaul_adm2_list'].notna()]

dfb_dr_adm2['gaul_adm2_list_int']=dfb_dr_adm2['gaul_adm2_list'].apply(lambda x: [int(i) for i in x])


dfb_dr_adm2['gaul_adm2_geob'] = dfb_dr_adm2['gaul_adm2_list_int'].apply(lambda x: [gaul_adm2_dict[i] for i in x])

#dfb_dr_a

dfb_dr_part2=dfb_dr_adm2[['Dis No','gaul_adm2_geob']]

dfb_dr_part2.columns=['Dis No','geob_adm2_list']
dfb_dr_part2

/tmp/ipykernel_45/449400531.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm2['gaul_adm2_list_int']=dfb_dr_adm2['gaul_adm2_list'].apply(lambda x: [int(i) for i in x])
/tmp/ipykernel_45/449400531.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr_adm2['gaul_adm2_geob'] = dfb_dr_adm2['gaul_adm2_list_int'].apply(lambda x: [gaul_adm2_dict[i] for i in x])


,Dis No,geob_adm2_list
14,1999-9388-ETH,"[ETH-ADM2-3_0_0-B45, ETH-ADM2-3_0_0-B47, ETH-A..."
15,1999-9388-KEN,"[KEN-ADM2-3_0_0-B50, KEN-ADM2-3_0_0-B52, KEN-A..."
18,1999-9388-RWA,"[RWA-ADM2-3_0_0-B7, RWA-ADM2-3_0_0-B10, RWA-AD..."
19,1999-9388-SOM,"[SOM-ADM2-3_0_0-B30, SOM-ADM2-3_0_0-B10, SOM-A..."
21,2004-9288-KEN,"[KEN-ADM2-3_0_0-B19, KEN-ADM2-3_0_0-B25, KEN-A..."
22,2003-9651-RWA,"[RWA-ADM2-3_0_0-B29, RWA-ADM2-3_0_0-B25, RWA-A..."
26,2005-9747-ETH,"[ETH-ADM2-3_0_0-B1, ETH-ADM2-3_0_0-B51, ETH-AD..."
27,2005-9213-DJI,"[DJI-ADM2-3_0_0-B10, DJI-ADM2-3_0_0-B9, DJI-AD..."
29,2005-9719-KEN,"[KEN-ADM2-3_0_0-B19, KEN-ADM2-3_0_0-B25, KEN-A..."
33,2008-9303-DJI,"[DJI-ADM2-3_0_0-B10, DJI-ADM2-3_0_0-B9, DJI-AD..."


# replacing manually added GEOB admin 1 and admin2 codes into common admin2 codes

In [8]:
dfb_dr1=dfb_dr[dfb_dr['geob'].notna()]
dfb_dr1['geob_list']=dfb_dr1['geob'].str.split(',')

def get_adm1_list(my_list):
    keys = ['ADM1','ADM2']
    aa=[[value for value in my_list if key in value] for key in set(keys)][0]
    bb=[[value for value in my_list if key in value] for key in set(keys)][1]
    #if not aa:
    #    bb=['']
    #else:
    #    bb=aa
    return aa,bb


aa_list=[]
bb_list=[]
for idx,row in dfb_dr1.iterrows():
    #dfb_dr1.loc[idx,'geob_adm2_list'] = get_adm1_list(row['geob_list'])
    aa,bb=get_adm1_list(row['geob_list'])
    #dfb_dr1.loc[idx,'geob_adm2_list'] =aa
    #print(aa)
    aa_list.append(aa)
    bb_list.append(bb)

dfb_dr1['geob_adm2_list']=aa_list
dfb_dr1['geob_adm1_list']=bb_list


# dfb_dr1['geob_adm2'] = dfb_dr1['geob_list'].apply(lambda x: [geob_db3_dict[i] for i in x])
dfb_dr1_adm1=dfb_dr1[['Dis No','geob_adm1_list']]
dfb_dr1_adm1_a=dfb_dr1_adm1[dfb_dr1_adm1['geob_adm1_list'].notna()]
dfb_dr1_adm1_a=dfb_dr1_adm1[dfb_dr1_adm1['geob_adm1_list'].map(lambda d: len(d)) > 0]
dfb_dr1_adm1_a['geob_adm2_list'] = dfb_dr1_adm1_a['geob_adm1_list'].apply(lambda x: [geob_db3_dict[i] for i in x])


def flatten(l):
    return [item for sublist in l for item in sublist]

aa_list=[]
for idx,row in dfb_dr1_adm1_a.iterrows():
    aa=flatten(row['geob_adm2_list'])
    aa_list.append(aa)
    #dfb_dr1_adm1_a.loc[idx,'geob_adm2_list_a'] = flatten(row['geob_adm2_list'])
    
dfb_dr1_adm1_a['geob_adm2_list_a']=aa_list

dfb_dr1_adm1_b=dfb_dr1_adm1_a[['Dis No','geob_adm2_list_a']]

dfb_dr2=pd.merge(dfb_dr1,dfb_dr1_adm1_b,on='Dis No',how='left')

dfb_dr2['geob_adm2_list_a'] = dfb_dr2['geob_adm2_list_a'].apply(lambda d: d if isinstance(d, list) else [])


geob_db1=geob_db[geob_db['shapeGroup']=='ERI']
eri_adm2_list=geob_db1['shapeID'].tolist()

geob_db1=geob_db[geob_db['shapeGroup']=='ETH']
eth_adm2_list=geob_db1['shapeID'].tolist()

geob_db1=geob_db[geob_db['shapeGroup']=='SDN']
sdn_adm2_list=geob_db1['shapeID'].tolist()

geob_db1=geob_db[geob_db['shapeGroup']=='SSD']
ssd_adm2_list=geob_db1['shapeID'].tolist()

old_sdn_list=sdn_adm2_list+ssd_adm2_list

geob_db1=geob_db[geob_db['shapeGroup']=='DJI']
dji_adm2_list=geob_db1['shapeID'].tolist()


dfb_dr2.at[1,'geob_adm2_list']=eri_adm2_list

dfb_dr2.at[4,'geob_adm2_list']=old_sdn_list

dfb_dr2.at[6,'geob_adm2_list']=dji_adm2_list

dfb_dr2.at[12,'geob_adm2_list']=eth_adm2_list

dfb_dr2['geob_adm2_list_a']=(dfb_dr2.geob_adm2_list+dfb_dr2.geob_adm2_list_a).map(set).map(list)

#from itertools import chain

#dfb_dr2['geob_adm2_list_a'] = [list(dict.fromkeys(chain(x, y))) for x, y in zip(dfb_dr2['geob_adm2_list'], dfb_dr2['geob_adm2_list_a'])]
#dfb_dr2.info()

dfb_dr_part3=dfb_dr2[['Dis No','geob_adm2_list_a']]

dfb_dr_part3.columns=['Dis No','geob_adm2_list']
dfb_dr_part3

/tmp/ipykernel_45/4091450130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr1['geob_list']=dfb_dr1['geob'].str.split(',')
/tmp/ipykernel_45/4091450130.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_dr1['geob_adm2_list']=aa_list
/tmp/ipykernel_45/4091450130.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Dis No,geob_adm2_list
0,1991-9224-KEN,"[KEN-ADM2-3_0_0-B65, KEN-ADM2-3_0_0-B4, KEN-AD..."
1,1993-9573-ERI,"[ERI-ADM2-3_0_0-B26, ERI-ADM2-3_0_0-B22, ERI-A..."
2,1990-9289-SDN,"[SSD-ADM2-3_0_0-B24, SSD-ADM2-3_0_0-B25, SSD-A..."
3,1990-9604-TZA,"[TZA-ADM2-3_0_0-B58, TZA-ADM2-3_0_0-B137, TZA-..."
4,1991-9686-SDN,"[SDN-ADM2-3_0_0-B120, SDN-ADM2-3_0_0-B141, SSD..."
5,1994-9422-KEN,"[KEN-ADM2-3_0_0-B44, KEN-ADM2-3_0_0-B24, KEN-A..."
6,1996-9119-DJI,"[DJI-ADM2-3_0_0-B1, DJI-ADM2-3_0_0-B8, DJI-ADM..."
7,1996-9326-KEN,"[KEN-ADM2-3_0_0-B44, KEN-ADM2-3_0_0-B61, KEN-A..."
8,1997-9091-ETH,"[ETH-ADM2-3_0_0-B30, ETH-ADM2-3_0_0-B4, ETH-AD..."
9,1996-9326-TZA,"[TZA-ADM2-3_0_0-B58, TZA-ADM2-3_0_0-B137, TZA-..."


In [9]:
dfb_parts=pd.concat([dfb_dr_part1,dfb_dr_part2,dfb_dr_part3],axis=0)
df=dfb_parts.groupby('Dis No').agg({'geob_adm2_list': 'sum'})
df['geob_adm2_list'] = df['geob_adm2_list'].apply(lambda x: list(set(x)))

#dfb_parts1=dfb_parts.drop_duplicates('Dis No')
#dfb_dr1=pd.merge(dfb_dr,dfb_parts1,on='Dis No',how='left')
#dfb_parts1.info()
#dfb_parts1.to_excel("em_dat_ea_drought_adm2.xlsx",index=False)  

In [10]:
df=dfb_parts.groupby('Dis No').agg({'geob_adm2_list': 'sum'})
#df.to_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_climada/ea-ibf-climada/impact_catalog/test1.csv')
df['geob_adm2_list1'] = df['geob_adm2_list'].apply(lambda x: list(set(x)))

In [11]:
df1=df[['geob_adm2_list1']]
df2=df1.reset_index()
df2.columns=['Dis No','geob_adm2_list']
df2.to_excel("em_dat_ea_drought_adm2_v2.xlsx",index=False)  

,Dis No,geob_adm2_list
0,1990-9289-SDN,"[SSD-ADM2-3_0_0-B24, SSD-ADM2-3_0_0-B26, SSD-A..."
1,1990-9604-TZA,"[TZA-ADM2-3_0_0-B4, TZA-ADM2-3_0_0-B45, TZA-AD..."
2,1991-9224-KEN,"[KEN-ADM2-3_0_0-B63, KEN-ADM2-3_0_0-B68, KEN-A..."
3,1991-9686-SDN,"[SDN-ADM2-3_0_0-B139, SDN-ADM2-3_0_0-B126, SDN..."
4,1993-9573-ERI,"[ERI-ADM2-3_0_0-B21, ERI-ADM2-3_0_0-B28, ERI-A..."
...,...,...
57,2021-9152-KEN,"[KEN-ADM2-3_0_0-B58, KEN-ADM2-3_0_0-B26, KEN-A..."
58,2021-9152-SOM,"[SOM-ADM2-3_0_0-B58, SOM-ADM2-3_0_0-B21, SOM-A..."
59,2021-9546-ETH,"[ETH-ADM2-3_0_0-B19, ETH-ADM2-3_0_0-B45, ETH-A..."
60,2021-9639-SSD,"[SSD-ADM2-3_0_0-B41, SSD-ADM2-3_0_0-B24, SSD-A..."
